In [2]:
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Other tools
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 41.6 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.8/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
# Load in state of the union address data
import pandas as pd
# Read in data
url = "/content/state_ofthe_union_texts.csv"
df = pd.read_csv(url)

# Subset df to modern era
modern = df[(df["Year"] >1945) & (df["Year"] < 2020)]

In [5]:
# Tokenize 

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
stop_words = stopwords.words('english')
stop_words.extend(['shall', 'applause', 'nation', 'year', 'must', 'world', 'people', 'america', 'american', 'us', 'congress','u', 'y'])
#stop_words.extend(['applause','year', 'u', 'y', 'shall'])
#stop_words.extend(['applause','year', 'u', 'y', 'per', 'cent', 'nof', 'nto', 'nthe', 'nand', 'nin', 'ni', 'nit', 'na'])
#stop_words.extend(['applause','year', 'u', 'y', 'per', 'cent', 'tonight', 'today'])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# compile sample documents into a list
# doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
doc_set = modern['Text'].values.tolist()

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)


# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=3, 
                                           id2word = dictionary, 
                                           passes=100)

for idx, topic in ldamodel.print_topics():
  print("Topic: {} \nWords: {}".format(idx, topic ))
  print("\n")
# print(ldamodel.print_topics(num_topics=10, num_words=10))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

Topic: 0 
Words: 0.006*"terrorist" + 0.006*"parent" + 0.005*"iraq" + 0.004*"compani" + 0.004*"republican" + 0.004*"terror" + 0.004*"medicar" + 0.003*"someth" + 0.003*"class" + 0.003*"immigr"


Topic: 1 
Words: 0.006*"communist" + 0.004*"oil" + 0.003*"vietnam" + 0.003*"1980" + 0.003*"strateg" + 0.003*"demonstr" + 0.003*"transport" + 0.003*"space" + 0.003*"object" + 0.002*"agricultur"


Topic: 2 
Words: 0.016*"expenditur" + 0.009*"estim" + 0.009*"1947" + 0.009*"agricultur" + 0.007*"appropri" + 0.007*"1946" + 0.007*"veteran" + 0.006*"adequ" + 0.006*"1945" + 0.005*"enterpris"




In [6]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3758757723690527


In [7]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -7.518868732121896


/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [8]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.152679  0.035772       1        1  42.654548
1      0.008895 -0.078621       2        1  41.177094
2      0.143784  0.042849       3        1  16.168358, topic_info=            Term        Freq       Total Category  logprob  loglift
433   expenditur  203.000000  203.000000  Default  30.0000  30.0000
15          1947   86.000000   86.000000  Default  29.0000  29.0000
417        estim  109.000000  109.000000  Default  28.0000  28.0000
95    agricultur  147.000000  147.000000  Default  27.0000  27.0000
1440   communist  160.000000  160.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
43             6   35.654555   85.928724   Topic3  -5.5980   0.9425
431       expans   34.378965   83.111741   Topic3  -5.6344   0.9394
1034      surplu   32.416997   74.507704   Topic3  -5.6932   0.9899
760      payment   33.217059   92.832362   Topic3  -5.6688   0.7944
637         loan   33.643437  110.159539   Topic3  -5.6561   0.6360

[223 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
13        1  0.052698     1945
13        3  0.930991     1945
14        3  0.985766     1946
15        3  0.988711     1947
16        3  0.890854     1948
...     ...       ...      ...
1135      1  0.033636  western
1135      2  0.824084  western
1135      3  0.134544  western
1661      2  0.949946    wheat
2327      1  0.977393       ye

[378 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])